# Load a target pixel file for an eclipsing binary

KIC 3954798 is an eclipsing binary with a very deep primary eclipse. Let's take a look at long cadence (30 minute) Kepler observations from the first Kepler quarter. We will load both the pipeline-reduced [light curve](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf#page=16) and the [target pixel file](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf#page=22) from MAST. 

* **Last Updated**: September 10, 2025
* **lcviz version**: 1.2.0

First we download the observations with `lightkurve`:

In [ ]:
from lcviz import LCviz
from lightkurve import search_lightcurve, search_targetpixelfile
from astropy.time import Time

# load a Kepler long-cadence light curve and TPF of a deep
# eclipsing binary with an orbital period of 0.91 days:
query = dict(
    target="KIC 3954798", 
    quarter=1, 
    author='Kepler', 
    cadence='long'
)

tpf = search_targetpixelfile(**query).download_all()[0]

# normalize the light curve by its median:
lc = search_lightcurve(**query).download_all()[0].normalize()

Now load both files of observations into LCviz:

In [ ]:
lcviz = LCviz()
lcviz.load(lc)
lcviz.load(tpf)
lcviz.show()

On the top, we see the light curve of the eclipsing binary. If you zoom in, you will see a deep primary eclipse and a shallower secondary eclipse. 

On the bottom (feel free to move the tab so they are left and right instead), we see a slice through the target pixel file (TPF). TPFs are data cubes containing a time series of small 2D images extracted from the Kepler field near the target. The image on the right shows the 2D image of the sky at the time selected in the `flux-vs-time` viewer, and marked with the vertical line.

We adjust the colormap on the image and launch the Time Selector plugin with the API:

In [ ]:
# adjust the colormap on the TPF image:
plot_opts = lcviz.plugins['Plot Options']
plot_opts.viewer = 'image'
plot_opts.reset_viewer_bounds()
plot_opts.stretch_vmin = 50
plot_opts.stretch_vmax = 22_000
plot_opts.stretch_function = 'arcsinh'

# open the Time Selector plugin:
time_sel = lcviz.plugins['Time Selector']
time_sel.open_in_tray()

You can select a different time to visualize in the `image` viewer by:
1. Selecting the second-to-last icon in the `flux-vs-time` viewer.
2. Clicking or dragging anywhere in the `flux-vs-time` viewer. The viewer on the right will update to show the spatial image at the new time.

You can also press the play/pause (⏯️) button in the Time Selector plugin to step through times in the TPF.

Now let's view the phase-folded light curve using the ephemeris from the [Villanova Kepler Eclipsing Binary Catalog](https://keplerebs.villanova.edu/):

In [ ]:
time_coordinates = lcviz.app.data_collection[0].coords
reference_time = time_coordinates.reference_time

period = 0.918480  # days
epoch = Time(54953.841706, format='mjd')

# phase-fold the transit light curve in an ephemeris viewer:
eph = lcviz.plugins['Ephemeris']
eph.period = period
eph.t0 = (
    (epoch - reference_time).to_value(time_coordinates.unit) % eph.period
)